In [ ]:
%pip install haystack-ai datasets sentence-transformer

In [ ]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from datasett import load_dataset
from haystack import Document
from haystack.components.embedders import SentenceTransformerDocumentEmbedder
from haystack.components.embedders import SentenceTransformerTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack import Pipeline
import os
from getpass import getpass
from haystack.components.generations import OpenAIGenerator

In [ ]:
document_store = InMemoryDocumentStore()

In [ ]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

In [ ]:
dataset

In [ ]:
for doc in dataset:
    print(doc["content"])

In [ ]:
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

In [ ]:
docs

In [ ]:
doc_embedder = SentenceTransformerDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [ ]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.wirte_documents(docs_with_embeddings["documents"])

In [ ]:
text_embedder = SentenceTransformerTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
retriever = InMemoryEmbeddingRetriever(document_store)

In [ ]:
template = """

Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content}}

{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")

In [ ]:
generator = OpenAIGenerator(model="gpt-3.5-turbo")

In [ ]:
basic_rag_pipeline = Pipeline()

# Add components to the pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", generator)

# Connect the component to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder.documents")
basic_rag_pipeline.connect("prompt_builder", "llm")

In [ ]:
quetsion = "What does Rhodes Statue look like?"

In [ ]:
response = basic_rag_pipeline.run({"text_embedder" : {"text": question}, "prompt_builder": {"question":quetsion}})

In [ ]:
response["llm"]["replies"][0]

In [ ]:
examples = [
"Where is Gardens of Babylon?",
"Why did people build Great Pyramid of Giza?",
"What does Rhodes Statue look like?",
"Why did people visit the Temple of Artemis?",
"What is the importance of Colossus of Rhodes?",
"What happened to the Tomb of Mausolus?",
"How did Colossus of Rhodes collapse?",
]

In [ ]:
question = """ Why did people visit the Temple of Artemis? """

In [ ]:
response = basic_rag_pipeline.run({"text_embedder" : {"text": quetsion}, "prompt_builder": {"question":quetsion}})

In [ ]:
response["llm"]["replies"][0]